In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; 
las; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

# [RAG 절차]

1. 문서를 읽는다
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다
    %pip install -qU langchain-text-splitters
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변 얻음(렝체인 사용 가능)
    %pip install -q langchain
    (https://smith.langchain.com에서 key생성 .env에 LANGCHAIN_API_KEY로 추가)

# 0. 패키지 설치

In [2]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [5]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [6]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 4.23 s
Wall time: 4.71 s


In [7]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters
## 2.1 1500토큰씩 쪼개서 읽어오기

In [ ]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter
loader = Docx2txtLoader('./data/소득세법(법률)(제21065호)(20260102).docx')
# gpt-4, gpt-4o, gpt-4 turbo, gpt4o-mini, embedding모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", #토큰을 세는 방식 이름
    chunk_size=1500,             # chunk 당 토큰 수 기준
    chunk_overlap=200
    # separators = ["\n", "\n\n"]파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 쪼개면서 읽는 시간 : ", runtime)

In [ ]:
len(documents) # chunk 수

In [ ]:
# chunk 글자수
# documents[0].page_content
print([len(document.page_content) for document in documents])

In [ ]:
# chunk 글자수 최대값, 최소값
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

## 2.2 1500글자 쪼개서 읽어오기

In [8]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader("data/소득세법(법률)(제21065호)(20260102).docx")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 1500글자씩 chunking
    chunk_overlap=200,
    # separators=["\n\n","\n"," ",""]
)
# 재귀적으로 다음 순서대로 시도:
# 1. \n\n(문단 구분)
# 2. \n(줄바꿈)
# 3. " "(공백)
# 4. "" 최후에는 글자 단위로 chunking
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 1500글자로 쪼개면서 읽는 시간 :", runtime)
print("chunk 갯수 :", len(documents))

문서를 1500글자로 쪼개면서 읽는 시간 : 4.321157693862915
chunk 갯수 : 193


In [9]:
print([len(document.page_content) for document in documents])

[1463, 1421, 1482, 1487, 1479, 1408, 1457, 1495, 1467, 1446, 1487, 1456, 1467, 1351, 1392, 1362, 1402, 1470, 1410, 1489, 1455, 1496, 1441, 1319, 1458, 1476, 1452, 1382, 1384, 1467, 1227, 1494, 1494, 1470, 1454, 1495, 1412, 1477, 1477, 1362, 1449, 1386, 1055, 1467, 1361, 1493, 1467, 1434, 1351, 1471, 1495, 1479, 1457, 1442, 1370, 873, 1419, 1357, 1353, 1316, 1349, 1452, 1439, 1363, 1433, 1412, 1306, 1200, 1411, 1452, 1421, 1318, 1416, 1333, 1308, 1385, 1479, 1495, 1399, 1375, 1360, 1353, 1382, 1446, 1356, 1409, 1483, 1486, 1157, 1233, 1443, 1474, 1369, 1439, 1451, 1495, 1443, 1489, 1484, 1407, 1432, 1436, 1468, 1442, 1477, 1396, 1423, 1282, 1496, 1486, 1376, 1342, 1466, 1385, 1491, 1477, 1470, 1385, 1477, 1445, 1485, 1373, 1495, 1443, 1419, 1456, 1451, 1305, 1454, 1411, 1443, 1488, 1404, 1419, 1339, 1451, 1288, 1450, 1481, 1419, 1369, 1479, 1480, 1461, 1414, 1419, 1463, 1481, 1486, 1387, 1485, 1448, 1367, 1364, 1391, 1446, 1414, 1414, 1414, 1473, 1417, 1474, 1419, 1342, 1406, 1338, 1138

In [10]:
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

1496
325


# 3. 쪼갠 문서를 임베딩 -> 벡터 베이터베이스 저장
- 임베딩 모델 : text-embedding-3-large (기본모델 :text-embedding-ada-002)
- 벡터데이터베이스(벡터 store) : chroma

In [11]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# embed_query() 한 문자열을 임베딩 벡터로 전환한 숫자 list를 return
len(embedding.embed_query("소득세법은 어쩌구"))

In [ ]:
embedding_vectors = embedding.embed_documents( # 여러 문자열을 임베딩 벡터로
    [
        documents[0].page_content,
        documents[1].page_content
    ]
)

In [ ]:
print(len(embedding_vectors), len(embedding_vectors[0]), len(embedding_vectors[1]))
print(embedding_vectors[0][:10])

In [ ]:
%%time
from langchain_chroma import Chroma
# 데이터 처음 저장할 때
# database = Chroma.from_documents(
#     documents=documents, # chunk
#     embedding=embedding, # 임베딩 객체
#     collection_name="tax-collection", # 생략시 이름 랜덤
#     persist_directory="./chroma"      # 생략시 로컬DB에 저장 안 됨. 프로그램 종료시 DB날라감
# )
# 이미 저장된 vector DB(store)를 사용할 때
database = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma"
)

In [ ]:
results = database._collection.get(include=['embeddings', 'documents', 'metadatas'])
print("데이터 수 :", len(results['ids']))
print("문서 임베딩 차원 수 :", len(results['embeddings'][0]))
print("1째 임베딩 샘플 :", results['embeddings'][1])
print("1번째 원본 :", results['documents'][1][:50])
print("1번째 metadata :", results['metadatas'][1])

# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query=query,
                                           k=2) # 기본 k값은 4

In [ ]:
#retrieved_docs

In [ ]:
# print("\n\n---\n\n".join([doc.page_content for doc in retrieved_docs]))
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 5. 유사도검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-1

In [ ]:
from langchain_openai import ChatOpenAI
load_dotenv()
llm = ChatOpenAI(model="gpt-4.1-nano")

In [ ]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_doc}
질문 : {query}"""

In [ ]:
ai_message = llm.invoke(prompt)

In [ ]:
ai_message.usage_metadata

In [ ]:
print(ai_message.content)

# 5. 유사도검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-2

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")
promptTemplate = ChatPromptTemplate([
    ("system", "당신은 최고의 한국 소득세 전문가입니다"),
    ("human", f"""다음 문맥을 참고하여 질문에 답변하세요.
    답을 모르면 모른다고 말하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{question}}
    문맥 : {{context}}
    답변 : """)
])
promptTemplate

In [ ]:
prompt = promptTemplate.invoke({
                            'context':retrieved_doc, # retrieved_docs보다 추천
                            'question':query
        })

In [ ]:
llm.invoke(prompt)

In [ ]:
# 위의 예제를 한번에
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(promptTemplate.invoke({'context':retrieved_doc,
                                                       'question':query})))

# 6. langchain